In [12]:
import MetaTrader5 as mt5
from datetime import datetime, timedelta, time
from IPython.display import display
import sys

# for jupyt notebooks
import os
d = os.path.dirname(os.path.dirname(os.getcwd()))
print(d)
sys.path.append(d)
#for python scripts
# from os.path import dirname, abspath
# d = dirname(dirname(abspath(__file__)))
# sys.path.append(d)

# print(sys.path)
# sys.path.append('ATJ/mt5_trade_utils')
from ATJ.mt5_trade_utils.backtester import Backtester, get_ohlc_history, create_price_fig, evaluate_backtest

c:\Users\derne\OneDrive - The Pennsylvania State University\Programming\Extra\algobot


In [13]:

if not mt5.initialize(): 
    print( 'initialization failed')
    quit() 
else: print('initialization success') 



login = 192918303
password = "Demo@1234"
server = "Exness-MT5Trial"

# mt5.login(login, password, server)


if not mt5.login(login, password, server):
    print("Failed to connect to the server")
else:
    print("Connected to the server successfully")
    print(mt5.account_info())


initialization success
Connected to the server successfully
AccountInfo(login=192918303, trade_mode=0, leverage=200, limit_orders=1024, margin_so_mode=0, trade_allowed=True, trade_expert=True, margin_mode=2, currency_digits=2, fifo_close=False, balance=9612.95, credit=0.0, profit=4890.13, equity=14503.08, margin=621.98, margin_free=13881.1, margin_level=2331.7598636612106, margin_so_call=60.0, margin_so_so=0.0, margin_initial=0.0, margin_maintenance=0.0, assets=0.0, liabilities=0.0, commission_blocked=0.0, name='DemoBotTester', server='Exness-MT5Trial', currency='USD', company='Exness Technologies Ltd')


In [16]:
symbol = 'EURUSDm'
start_dt = datetime(2023, 1, 1)
end_dt = datetime.now()

# get historical data
mt5.initialize()
ohlc = get_ohlc_history(symbol, mt5.TIMEFRAME_D1, start_dt, end_dt)

sma_period = 20

ohlc['sma_20'] = ohlc['open'].rolling(sma_period).mean()
ohlc['std'] = ohlc['open'].rolling(sma_period).std()
ohlc['upper_band'] = ohlc['sma_20'] + 1.5 * ohlc['std']
ohlc['lower_band'] = ohlc['sma_20'] - 1.5 * ohlc['std']


display(ohlc)

fig = create_price_fig(ohlc, indicators=['sma_20', 'upper_band', 'lower_band'])
display(fig)

,time,open,high,low,close,sma_20,std,upper_band,lower_band
0,2023-01-02,1.06978,1.07066,1.06357,1.06683,NaN,NaN,NaN,NaN
1,2023-01-03,1.06686,1.06797,1.05192,1.05452,NaN,NaN,NaN,NaN
2,2023-01-04,1.05453,1.06351,1.05449,1.06058,NaN,NaN,NaN,NaN
3,2023-01-05,1.06056,1.06311,1.05148,1.05204,NaN,NaN,NaN,NaN
4,2023-01-06,1.05206,1.06476,1.04791,1.06443,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
640,2025-01-20,1.02800,1.04343,1.02777,1.04206,1.031866,0.005688,1.040397,1.023334
641,2025-01-21,1.04206,1.04352,1.03413,1.04054,1.031834,0.005625,1.040271,1.023396
642,2025-01-22,1.04053,1.04569,1.03994,1.04074,1.031714,0.005398,1.039811,1.023617
643,2025-01-23,1.04074,1.04375,1.03715,1.04153,1.031725,0.005417,1.039850,1.023600


In [17]:
def get_signal(x):
    if x['open'] > x['upper_band']:
        return 'sell'
    elif x['open'] < x['lower_band']:
        return 'buy'
    
def get_exit_signal(x):
    if x['open'] > x['sma_20']:
        return 'close buy positions'
    elif x['open'] < x['lower_band']:
        return 'close sell positions'
    
ohlc['signal'] = ohlc.apply(get_signal, axis=1)
ohlc['exit_signal'] = ohlc.apply(get_exit_signal, axis=1)
ohlc

,time,open,high,low,close,sma_20,std,upper_band,lower_band,signal,exit_signal
0,2023-01-02,1.06978,1.07066,1.06357,1.06683,NaN,NaN,NaN,NaN,None,None
1,2023-01-03,1.06686,1.06797,1.05192,1.05452,NaN,NaN,NaN,NaN,None,None
2,2023-01-04,1.05453,1.06351,1.05449,1.06058,NaN,NaN,NaN,NaN,None,None
3,2023-01-05,1.06056,1.06311,1.05148,1.05204,NaN,NaN,NaN,NaN,None,None
4,2023-01-06,1.05206,1.06476,1.04791,1.06443,NaN,NaN,NaN,NaN,None,None
...,...,...,...,...,...,...,...,...,...,...,...
640,2025-01-20,1.02800,1.04343,1.02777,1.04206,1.031866,0.005688,1.040397,1.023334,None,None
641,2025-01-21,1.04206,1.04352,1.03413,1.04054,1.031834,0.005625,1.040271,1.023396,sell,close buy positions
642,2025-01-22,1.04053,1.04569,1.03994,1.04074,1.031714,0.005398,1.039811,1.023617,sell,close buy positions
643,2025-01-23,1.04074,1.04375,1.03715,1.04153,1.031725,0.005417,1.039850,1.023600,sell,close buy positions


In [18]:
# create trade logic
def on_bar(data, trades, orders):
    volume = 100000 # 1 lots
    
    open_trades = trades[trades['state'] == 'open']
    num_open_trades = open_trades.shape[0]
    
    # entry signal
    if data['signal'] == 'buy' and not num_open_trades:
        orders.open_trade(symbol, volume, 'buy')
    
    elif data['signal'] == 'sell' and not num_open_trades:
        orders.open_trade(symbol, volume, 'sell')
        
    # exit signal
    if num_open_trades:
        trade = open_trades.iloc[0]

        if trade['order_type'] == 'buy' and data['exit_signal'] == 'close buy positions':
            orders.close_trade(trade)
        elif trade['order_type'] == 'sell' and data['exit_signal'] == 'close sell positions':
            orders.close_trade(trade)

In [20]:
# backtest parameters
starting_balance = 10000
currency = 'USD'
exchange_rate = 1
commission = -7 / 100000

# backtest
bt = Backtester()
bt.set_starting_balance(starting_balance, currency=currency)
bt.set_exchange_rate(exchange_rate)
bt.set_commission(commission)

bt.set_historical_data(ohlc)
bt.set_on_bar(on_bar)

bt.run_backtest()

bt.trades

,state,symbol,order_type,volume,open_time,open_price,close_time,close_price,sl,tp,info,profit,commission,profit_net,profit_cumulative,balance
0,closed,EURUSDm,sell,100000,2023-02-02 00:00:00,1.10123,2023-02-07 00:00:00,1.07285,0,0,{},2838.0,-7.0,2831.0,2831.0,12831.0
1,closed,EURUSDm,buy,100000,2023-02-08 00:00:00,1.07297,2023-03-02 00:00:00,1.06682,0,0,{},-615.0,-7.0,-622.0,2209.0,12209.0
2,closed,EURUSDm,buy,100000,2023-03-08 00:00:00,1.05479,2023-03-12 00:00:00,1.06727,0,0,{},1248.0,-7.0,1241.0,3450.0,13450.0
3,closed,EURUSDm,sell,100000,2023-03-14 00:00:00,1.07228,2023-05-12 00:00:00,1.09121,0,0,{},-1893.0,-7.0,-1900.0,1550.0,11550.0
4,closed,EURUSDm,buy,100000,2023-05-14 00:00:00,1.08511,2023-06-09 00:00:00,1.07815,0,0,{},-696.0,-7.0,-703.0,847.0,10847.0
5,closed,EURUSDm,sell,100000,2023-06-14 00:00:00,1.07891,2023-08-03 00:00:00,1.09431,0,0,{},-1540.0,-7.0,-1547.0,-700.0,9300.0
6,closed,EURUSDm,buy,100000,2023-08-15 00:00:00,1.09073,2023-08-31 00:00:00,1.09292,0,0,{},219.0,-7.0,212.0,-488.0,9512.0
7,closed,EURUSDm,buy,100000,2023-09-03 00:00:00,1.07756,2023-10-11 00:00:00,1.06039,0,0,{},-1717.0,-7.0,-1724.0,-2212.0,7788.0
8,closed,EURUSDm,sell,100000,2023-10-24 00:00:00,1.06696,2023-12-07 00:00:00,1.07659,0,0,{},-963.0,-7.0,-970.0,-3182.0,6818.0
9,closed,EURUSDm,buy,100000,2023-12-08 00:00:00,1.07895,2023-12-14 00:00:00,1.08821,0,0,{},926.0,-7.0,919.0,-2263.0,7737.0


In [23]:
pnl_chart = bt.plot_pnl()
pnl_chart

In [24]:
backtest_fig = bt.visualize_backtest(indicators=['sma_20', 'upper_band', 'lower_band'])
backtest_fig

In [25]:
bt.export_to_json('backtest_example.json', indicators=['upper_band', 'lower_band'])

1

In [26]:
evaluate_backtest(bt.trades)

biggest_profit 2838.0
biggest_loss -1893.0


,state,symbol,order_type,volume,open_time,open_price,close_time,close_price,sl,tp,info,profit,commission,profit_net,profit_cumulative,balance
0,closed,EURUSDm,sell,100000,2023-02-02,1.10123,2023-02-07,1.07285,0,0,{},2838.0,-7.0,2831.0,2831.0,12831.0
2,closed,EURUSDm,buy,100000,2023-03-08,1.05479,2023-03-12,1.06727,0,0,{},1248.0,-7.0,1241.0,3450.0,13450.0
6,closed,EURUSDm,buy,100000,2023-08-15,1.09073,2023-08-31,1.09292,0,0,{},219.0,-7.0,212.0,-488.0,9512.0
9,closed,EURUSDm,buy,100000,2023-12-08,1.07895,2023-12-14,1.08821,0,0,{},926.0,-7.0,919.0,-2263.0,7737.0
10,closed,EURUSDm,sell,100000,2023-12-22,1.10051,2024-01-17,1.0878,0,0,{},1271.0,-7.0,1264.0,-999.0,9001.0
12,closed,EURUSDm,sell,100000,2024-02-27,1.08499,2024-03-24,1.08067,0,0,{},432.0,-7.0,425.0,-1055.0,8945.0
13,closed,EURUSDm,buy,100000,2024-03-25,1.0807,2024-04-07,1.08362,0,0,{},292.0,-7.0,285.0,-770.0,9230.0
17,closed,EURUSDm,sell,100000,2024-07-04,1.07905,2024-08-02,1.0786,0,0,{},45.0,-7.0,38.0,-1348.0,8652.0
19,closed,EURUSDm,buy,100000,2024-09-12,1.101,2024-09-17,1.11286,0,0,{},1186.0,-7.0,1179.0,-873.0,9127.0
20,closed,EURUSDm,sell,100000,2024-09-20,1.11589,2024-10-04,1.10346,0,0,{},1243.0,-7.0,1236.0,363.0,10363.0


,state,symbol,order_type,volume,open_time,open_price,close_time,close_price,sl,tp,info,profit,commission,profit_net,profit_cumulative,balance
1,closed,EURUSDm,buy,100000,2023-02-08,1.07297,2023-03-02,1.06682,0,0,{},-615.0,-7.0,-622.0,2209.0,12209.0
3,closed,EURUSDm,sell,100000,2023-03-14,1.07228,2023-05-12,1.09121,0,0,{},-1893.0,-7.0,-1900.0,1550.0,11550.0
4,closed,EURUSDm,buy,100000,2023-05-14,1.08511,2023-06-09,1.07815,0,0,{},-696.0,-7.0,-703.0,847.0,10847.0
5,closed,EURUSDm,sell,100000,2023-06-14,1.07891,2023-08-03,1.09431,0,0,{},-1540.0,-7.0,-1547.0,-700.0,9300.0
7,closed,EURUSDm,buy,100000,2023-09-03,1.07756,2023-10-11,1.06039,0,0,{},-1717.0,-7.0,-1724.0,-2212.0,7788.0
8,closed,EURUSDm,sell,100000,2023-10-24,1.06696,2023-12-07,1.07659,0,0,{},-963.0,-7.0,-970.0,-3182.0,6818.0
11,closed,EURUSDm,buy,100000,2024-01-24,1.08566,2024-02-21,1.08092,0,0,{},-474.0,-7.0,-481.0,-1480.0,8520.0
14,closed,EURUSDm,buy,100000,2024-04-11,1.07409,2024-04-26,1.07264,0,0,{},-145.0,-7.0,-152.0,-922.0,9078.0
15,closed,EURUSDm,sell,100000,2024-05-05,1.07608,2024-06-09,1.07621,0,0,{},-13.0,-7.0,-20.0,-942.0,9058.0
16,closed,EURUSDm,buy,100000,2024-06-10,1.07782,2024-07-01,1.07345,0,0,{},-437.0,-7.0,-444.0,-1386.0,8614.0


avg_win 1035.4166666666667
avg_loss -866.7142857142857
count_profit_trades 12
count_loss_trades 14
win_rate 0.86
rrr 1.19


,order_type,profit
0,buy,-1565.0
1,sell,1856.0


,state,symbol,order_type,volume,open_time,open_price,close_time,close_price,sl,tp,info,profit,commission,profit_net,profit_cumulative,balance,dayofweek
0,closed,EURUSDm,sell,100000,2023-02-02,1.10123,2023-02-07,1.07285,0,0,{},2838.0,-7.0,2831.0,2831.0,12831.0,1
1,closed,EURUSDm,buy,100000,2023-02-08,1.07297,2023-03-02,1.06682,0,0,{},-615.0,-7.0,-622.0,2209.0,12209.0,3
2,closed,EURUSDm,buy,100000,2023-03-08,1.05479,2023-03-12,1.06727,0,0,{},1248.0,-7.0,1241.0,3450.0,13450.0,6
3,closed,EURUSDm,sell,100000,2023-03-14,1.07228,2023-05-12,1.09121,0,0,{},-1893.0,-7.0,-1900.0,1550.0,11550.0,4
4,closed,EURUSDm,buy,100000,2023-05-14,1.08511,2023-06-09,1.07815,0,0,{},-696.0,-7.0,-703.0,847.0,10847.0,4
5,closed,EURUSDm,sell,100000,2023-06-14,1.07891,2023-08-03,1.09431,0,0,{},-1540.0,-7.0,-1547.0,-700.0,9300.0,3
6,closed,EURUSDm,buy,100000,2023-08-15,1.09073,2023-08-31,1.09292,0,0,{},219.0,-7.0,212.0,-488.0,9512.0,3
7,closed,EURUSDm,buy,100000,2023-09-03,1.07756,2023-10-11,1.06039,0,0,{},-1717.0,-7.0,-1724.0,-2212.0,7788.0,2
8,closed,EURUSDm,sell,100000,2023-10-24,1.06696,2023-12-07,1.07659,0,0,{},-963.0,-7.0,-970.0,-3182.0,6818.0,3
9,closed,EURUSDm,buy,100000,2023-12-08,1.07895,2023-12-14,1.08821,0,0,{},926.0,-7.0,919.0,-2263.0,7737.0,3


,state,symbol,order_type,volume,open_time,open_price,close_time,close_price,sl,tp,info,profit,commission,profit_net,profit_cumulative,balance,dayofweek,current_max,drawdown
0,closed,EURUSDm,sell,100000,2023-02-02,1.10123,2023-02-07,1.07285,0,0,{},2838.0,-7.0,2831.0,2831.0,12831.0,1,2831.0,0.0
1,closed,EURUSDm,buy,100000,2023-02-08,1.07297,2023-03-02,1.06682,0,0,{},-615.0,-7.0,-622.0,2209.0,12209.0,3,2831.0,-622.0
2,closed,EURUSDm,buy,100000,2023-03-08,1.05479,2023-03-12,1.06727,0,0,{},1248.0,-7.0,1241.0,3450.0,13450.0,6,3450.0,0.0
3,closed,EURUSDm,sell,100000,2023-03-14,1.07228,2023-05-12,1.09121,0,0,{},-1893.0,-7.0,-1900.0,1550.0,11550.0,4,3450.0,-1900.0
4,closed,EURUSDm,buy,100000,2023-05-14,1.08511,2023-06-09,1.07815,0,0,{},-696.0,-7.0,-703.0,847.0,10847.0,4,3450.0,-2603.0
5,closed,EURUSDm,sell,100000,2023-06-14,1.07891,2023-08-03,1.09431,0,0,{},-1540.0,-7.0,-1547.0,-700.0,9300.0,3,3450.0,-4150.0
6,closed,EURUSDm,buy,100000,2023-08-15,1.09073,2023-08-31,1.09292,0,0,{},219.0,-7.0,212.0,-488.0,9512.0,3,3450.0,-3938.0
7,closed,EURUSDm,buy,100000,2023-09-03,1.07756,2023-10-11,1.06039,0,0,{},-1717.0,-7.0,-1724.0,-2212.0,7788.0,2,3450.0,-5662.0
8,closed,EURUSDm,sell,100000,2023-10-24,1.06696,2023-12-07,1.07659,0,0,{},-963.0,-7.0,-970.0,-3182.0,6818.0,3,3450.0,-6632.0
9,closed,EURUSDm,buy,100000,2023-12-08,1.07895,2023-12-14,1.08821,0,0,{},926.0,-7.0,919.0,-2263.0,7737.0,3,3450.0,-5713.0


ValueError: Plotly Express cannot process wide-form data with columns of different type.

In [27]:
bt.trades['commission'] = bt.trades['commission'].round(2)
bt.trades.to_csv('bt_trades.csv', index=False)